In [1]:
library("tidyverse")

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v ggplot2 3.1.1       v purrr   0.3.2  
v tibble  2.1.1       v dplyr   0.8.0.1
v tidyr   0.8.3       v stringr 1.4.0  
v readr   1.3.1       v forcats 0.4.0  
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


In [2]:
library("DT")

Warning message:
"package 'DT' was built under R version 3.6.3"

In [3]:
cases <- read_csv("COVID-19_cases_plus_census.csv")

ERROR: Error: 'COVID-19_cases_plus_census.csv' does not exist in current working directory ('C:/Users/henry/R Projects/DM/Project 3').


In [ ]:
cases <- cases %>% mutate_if(is.character, factor)
dim(cases)

In [ ]:
cases <- cases %>% filter(confirmed_cases > 0) 

cases <- cases %>% 
  arrange(desc(confirmed_cases)) #%>%    
  #select(county_name, state, confirmed_cases, deaths, total_pop, median_income, median_age)
cases <- cases %>% mutate(
  cases_per_10000 = confirmed_cases/total_pop*10000, 
  deaths_per_10000 = deaths/total_pop*10000, 
  death_per_case = deaths/confirmed_cases)

cases

In [ ]:
cases_sel <- cases %>% select(county_name, state, total_pop,
  nonfamily_households, median_year_structure_built,        
  female_pop, median_age, white_pop, 
  black_pop, asian_pop, hispanic_pop, amerindian_pop,
  commuters_by_public_transportation, 
  households, median_income, housing_units, 
  vacant_housing_units, 
  percent_income_spent_on_rent,
  employed_pop, unemployed_pop, 
  in_school, in_undergrad_college,
  cases_per_10000, deaths_per_10000, death_per_case)

# normalize by population 
cases_sel <- cases_sel %>% mutate(
  nonfamily_households = nonfamily_households / total_pop, 
  female_pop = female_pop / total_pop,
  white_pop = white_pop / total_pop, 
  black_pop = black_pop / total_pop, 
  asian_pop = asian_pop / total_pop, 
  hispanic_pop = hispanic_pop / total_pop, 
  amerindian_pop = amerindian_pop / total_pop,
  commuters_by_public_transportation = commuters_by_public_transportation/ total_pop, 
  households = households / total_pop, 
  housing_units = housing_units / total_pop, 
  vacant_housing_units = vacant_housing_units / total_pop, 
  employed_pop = employed_pop / total_pop, 
  unemployed_pop = unemployed_pop / total_pop, 
  in_school = in_school / total_pop, 
  in_undergrad_college = in_undergrad_college / total_pop 
  )

cases_sel

In [ ]:
table(complete.cases(cases_sel))

In [4]:
str(cases_sel)

ERROR: Error in str(cases_sel): object 'cases_sel' not found


In [5]:
library(seriation)
cm <- cor(cases_sel %>% select_if(is.numeric) %>% na.omit)

Warning message:
"package 'seriation' was built under R version 3.6.3"

ERROR: Error in eval(lhs, parent, parent): object 'cases_sel' not found


In [6]:
hmap(cm, margins = c(14,14))

ERROR: Error in as.vector(x, mode): cannot coerce type 'closure' to vector of type 'any'


In [ ]:
cases_sel <- cases_sel %>% mutate(bad = as.factor(deaths_per_10000 > 10))

In [ ]:
cases_sel %>% pull(bad) %>% table()

In [ ]:
cases_sel %>% group_by(state) %>% 
  summarize(bad_pct = sum(bad == TRUE)/n()) %>%
  arrange(desc(bad_pct))

In [ ]:
cases_train <- cases_sel %>% filter(state %in% c("TX", "CA", "FL", "NY"))
cases_train %>% pull(bad) %>% table()

In [ ]:
cases_test <-  cases_sel %>% filter(!(state %in% c("TX", "CA", "FL", "NY")))
cases_test %>% pull(bad) %>% table()

In [ ]:
counties <- as_tibble(map_data("county"))
counties <- counties %>% 
  rename(c(county = subregion, state = region)) %>%
  mutate(state = state.abb[match(state, tolower(state.name))]) %>%
  select(state, county, long, lat, group)
counties  

In [ ]:
counties_all <- counties %>% left_join(cases_train %>% 
    mutate(county = county_name %>% str_to_lower() %>% 
        str_replace('\\s+county\\s*$', '')))

In [ ]:
ggplot(counties_all, aes(long, lat)) + 
  geom_polygon(aes(group = group, fill = bad), color = "black", size = 0.1) + 
    coord_quickmap() + scale_fill_manual(values = c('TRUE' = 'red', 'FALSE' = 'grey'))

In [ ]:
library(FSelector)
cases_train %>%  chi.squared(bad ~ ., data = .) %>% 
  arrange(desc(attr_importance)) %>% head()

In [ ]:
cases_train <- cases_train %>% select(-c(deaths_per_10000))
cases_train %>%  chi.squared(bad ~ ., data = .) %>% 
  arrange(desc(attr_importance)) %>% head()

In [ ]:
cases_train <- cases_train %>% select(-death_per_case, -cases_per_10000)

cases_train %>%  chi.squared(bad ~ ., data = .) %>% 
  arrange(desc(attr_importance)) %>% head(n = 10)

In [ ]:
library(caret)

In [ ]:
fit <- cases_train %>%
  train(bad ~ . - county_name - state,
    data = . ,
    #method = "rpart",
    method = "rf",
    #method = "nb",
    trControl = trainControl(method = "cv", number = 10)
    )
fit

In [ ]:
#library(rpart.plot)
#rpart.plot(fit$finalModel, extra = 2)

varImp(fit)

In [ ]:
cases_test <- cases_test %>% na.omit

In [ ]:
cases_test$bad_predicted <- predict(fit, cases_test)

In [ ]:
counties_test <- counties %>% left_join(cases_test %>% 
    mutate(county = county_name %>% str_to_lower() %>% 
        str_replace('\\s+county\\s*$', '')))

In [ ]:
ggplot(counties_test, aes(long, lat)) + 
  geom_polygon(aes(group = group, fill = bad), color = "black", size = 0.1) + 
  coord_quickmap() + 
  scale_fill_manual(values = c('TRUE' = 'red', 'FALSE' = 'grey'))

In [ ]:
ggplot(counties_test, aes(long, lat)) + 
  geom_polygon(aes(group = group, fill = bad_predicted), color = "black", size = 0.1) + 
    coord_quickmap() + 
  scale_fill_manual(values = c('TRUE' = 'red', 'FALSE' = 'grey'))

In [ ]:
confusionMatrix(data = cases_test$bad_predicted, ref = cases_test$bad)